### 图像卷积

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

#### 自定义一个单层卷积

In [55]:
import torch.nn.functional as F
def Conv(X, kernel, stride = 1, padding = 0):
    h_in, w_in = X.shape
    k_weights = torch.randn((kernel, kernel))
    h_out = (h_in + 2*padding - kernel) // stride + 1
    w_out = (w_in + 2*padding - kernel) // stride + 1
    Y = torch.zeros((h_out, w_out))
    if padding:
        pad = nn.ZeroPad2d(padding)
        X = pad(X)
    for i in range(h_out):
        for j in range(w_out):
            Y[i, j] = (X[i*stride:i*stride + kernel, j*stride:j*stride+kernel] * k_weights).sum()
    return Y


In [57]:
X = torch.arange(64).reshape(8,8)
kernel = 3
Conv(X, kernel, padding=1, stride = 2).shape

torch.Size([4, 4])

#### 简单定义二维卷积

In [60]:
X = torch.ones((12, 12))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.]])

In [64]:
# 设置卷积核
K = torch.tensor([[1.0, -1.0],
                  [1.0, -1.0],
                  [1.0, -1.0]])

torch.Size([3, 2])

In [68]:
# 通过卷积检测到边缘
Y = corr2d(X, K)

tensor([[ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.]])

#### 学习由`X`生成`Y`的卷积核

In [69]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))